## AB Tests

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# загружаю руками каждый файл, предварительно зазиповав
from google.colab import files
uploaded = files.upload()

Saving dataset_part1_students.zip to dataset_part1_students.zip
Saving dataset_part2_students.zip to dataset_part2_students.zip
Saving dataset_part3_students.zip to dataset_part3_students.zip


In [ ]:
ds1 = pd.read_csv('/content/dataset_part1_students.zip', compression='zip', header=0, sep='\t', quotechar='"')
ds2 = pd.read_csv('/content/dataset_part2_students.zip', compression='zip', header=0, sep='\t', quotechar='"')
ds3 = pd.read_csv('/content/dataset_part3_students.zip', compression='zip', header=0, sep='\t', quotechar='"')

### 1. Оценка результатов AB теста

В данном задании вам требуется оценить результаты AB теста. Даны результаты на контрольной и тестовой группе после проведения AB теста.
Помимо показателей целевой переменной, вам даны также несколько фичей, которые являются характеристиками объектов. Даны целевые показатели (y) и принадлежность к целевой группе (W).

</u>Задача:</u>
1. Проверьте подходящую для данных гипотезу о равенстве средних в выборках
2. Каков treatment effect нашего воздействия? (ATE) 
3. Примените известные вам методы оценки ATE с учетом особенностей данных.
4. Проанализируйте, сравните результаты применения различных методов.
5. В явном виде выпишите выводы.

</i>Примечание: предположения о рандомизированном эксперименте нуждаются в проверке - независимость treatment от объектов не гарантирована. Все предположения выписывайте явно.</i> 

Для данной задачи используется dataset:  dataset_part1_students.csv

In [ ]:
ds1.head()

,y,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,-1.459404,0.0,0.986277,0.873392,0.509746,0.271836,0.336919,0.216954,0.276477,0.343316,0.862159,0.156700
1,-1.694174,0.0,0.140887,0.757080,0.736325,0.355663,0.341093,0.666803,0.217101,0.561427,0.124179,0.319736
2,5.378374,0.0,0.953214,0.137357,0.569413,0.975665,0.503367,0.667664,0.034191,0.456119,0.155851,0.476049
3,2.188189,1.0,0.169702,0.896258,0.373394,0.379693,0.858317,0.646061,0.583462,0.668350,0.177793,0.849248
4,2.862696,1.0,0.442373,0.831468,0.763921,0.919691,0.070573,0.156165,0.636894,0.555696,0.191929,0.425656


 1.1) Разбиваю датасет на 2 подвыборки: treatment и control

In [ ]:
tr1 = ds1[ds1['W'] == 1]
cn1 = ds1[ds1['W'] == 0]

1.2) Дальше хочу использовать Z критерий, чтобы проверить гипотезу о равенстве средних выборок, но сразу я его применить не могу, потому что не знаю, нормально ли распределены выборки. Проверяю выборки на нормальность с помощью теста Шапиро-Уилка, информацию нашел тут: https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/ . Взял этот критерий, потому что уже есть его библиотечная реализация. (критерий хи квадрат не нагуглил, поэтому остановился на этом)

H0: распределение - нормальное  
Н1: распределение - любое кроме нормального  
Задаюсь уровнем значимости alpha = 0.05

In [ ]:
from scipy.stats import shapiro

In [ ]:
stat, p = shapiro(tr1['y'])
print('Statistics={},p={}'.format(stat, p))

Statistics=0.9998707175254822,p=0.8599085807800293


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
stat, p = shapiro(cn1['y'])
print('Statistics={},p={}'.format(stat, p))

Statistics=0.9998553991317749,p=0.837329626083374


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Р больше уровня значимости, то есть нет оснований отвергать нулевую гипотезу о нормальности распределения каждой выборки.

1.3) Теперь могу использовать Z-тест, тк предположение о нормальности распределения выборок выполняется.

H0: средние выборок равны  
Н1: средние выборок не равны  
Задаюсь уровнем значимости alpha = 0.05  

In [ ]:
from statsmodels.stats.weightstats import ztest

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
ztest(tr1['y'], cn1['y'])

(9.332400850259313, 1.0349941796938945e-20)

P много меньше уровня значимости aplha, поэтому отвергаю гипотезу Н0 о равенстве средних.

2,3) Treatment effect воздействия и его оценки

В лекции сказано,что ATE может быть вычислен как среднее от разности таргета объекта с воздействием и таргета объекта без воздействия. Но для того, чтобы использовать эту формулу, нужно, чтобы эксперимет был рандомизирован. В рандомизированном эксперименте таргет не зависит от воздействия (в нашем случае это не так, тк средние выборок не равны). Тем не менее я не уверен, что случайность эксперимента проверяется по критерию, указанному выше, поэтому я посчитаю АТЕ как для рандомизированного эксперимента (хотя скорее всего он таким не является).

In [ ]:
ATE_rnd = tr1['y'].mean()-cn1['y'].mean() 

In [ ]:
ATE_rnd

0.4001031055444413

Так же можно найти ATE как коэффициент перед W в модели линейной регрессии.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = ds1.drop('y', axis=1)
y = ds1['y']
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
X.head(1)

,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,0.0,0.986277,0.873392,0.509746,0.271836,0.336919,0.216954,0.276477,0.343316,0.862159,0.1567


In [ ]:
ATE_linreg = model.coef_[0]

In [ ]:
ATE_linreg

0.5133854882565612

4) Получил 2 разных значения оценки, что логично, потому что первый способ вычисления основан на том, что W не зависит от (Y(0),Y(1)). Думаю,что такая офенка АТЕ неверна. Второе значение получено с учетом остальных фичей, тк получено как коэффициент из модели линейной регрессии. При втором решении учитывается что W зависит от (Y(0),Y(1)), поэтому такая оценка точнее.

In [ ]:
ATE_rnd

0.4001031055444413

In [ ]:
ATE_linreg

0.5133854882565612

5) Вывод: оценка, полученная как коэффициент модели линейной регрессии точнее, потому что не опирается на предположение, что эксперимент рандомизирован (что скорее всего не так).

### 2. Оценка результатов AB теста с показателями до начала эксперимента

Необходимо как и ранее оценить результаты AB теста, а именно ATE. Вы располагаете результатами до и после проведения теста (y_0 и y_1), а также вам дан treatment assignment (W). 

Необходимо: 
1. Проверить все требуемые гипотезы
2. Оценить ATE подходящим для этого случая методом (примените несколько методов)
3. Сравнить и проанализировать результаты оценок
4. Явно выписать выводы.

Как и прежде, все предположения должны быть указаны. Визуализация результатов приветствуется.

dataset_part2_students.csv

In [ ]:
ds2.head()

,y_1,y_0,W,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,5.887156,2.785060,1.0,0.605978,0.733369,0.138947,0.312673,0.997243,0.128162,0.178993,0.752925,0.662161,0.784310
1,2.232601,5.533515,0.0,0.096894,0.058571,0.962396,0.616557,0.086630,0.561272,0.616525,0.963843,0.574304,0.371161
2,0.144419,0.095546,1.0,0.452145,0.201850,0.569305,0.195096,0.583704,0.476313,0.517814,0.823099,0.732225,0.069056
3,-2.446425,3.590472,1.0,0.672129,0.643485,0.828014,0.204469,0.617489,0.617701,0.301069,0.871741,0.589654,0.981770
4,2.163090,1.365405,0.0,0.442232,0.126318,0.508831,0.431786,0.915940,0.709016,0.890655,0.588886,0.636830,0.342209


1) Проверка гипотез

Добавляю новую фичу - разность между у1 и у0.

In [ ]:
ds2['delt'] = ds2['y_1']-ds2['y_0']

Далее так же делю на treatment и control.


In [ ]:
tr2 = ds2[ds2['W'] == 1]
cn2 = ds2[ds2['W'] == 0]

Аналогично первому заданию сначала проверю,что выборки распределены нормально, дальше с помощью Zтеста проверю гипотезу у равенстве средних.

H0: распределение - нормальное  
Н1: распределение - любое кроме нормального  
Задаюсь уровнем значимости alpha = 0.05

In [ ]:
stat, p = shapiro(tr2['delt'])
print('Statistics={},p={}'.format(stat, p))

Statistics=0.9996042251586914,p=0.38465145230293274


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
stat, p = shapiro(cn2['delt'])
print('Statistics={},p={}'.format(stat, p))

Statistics=0.9994486570358276,p=0.17446228861808777


Р больше уровня значимости, то есть нет оснований отвергать нулевую гипотезу о нормальности распределения каждой выборки.

H0: средние выборок равны  
Н1: средние выборок не равны  
Задаюсь уровнем значимости alpha = 0.05  

In [ ]:
ztest(tr2['delt'], cn2['delt'])

(9.118025905018952, 7.650513264824386e-20)

P много меньше уровня значимости aplha, поэтому отвергаю гипотезу Н0 о равенстве средних.

2) Оценка АТЕ

Для оценки АТЕ применяю методы DiD, CUPED, ANCOVA

2.1) DiD

Использую формулу y = betI(T=1)+gamI(W=1)+tetI(T=1)I(W=1)  Пересоберу датасет,чтобы он соответствовал формуле.

In [ ]:
#первый датасет, Т=0
d1 = ds2[['y_0','W']]
d1.columns = ['Y','W']
d1['T'] = 0
d1['WT'] = d1['T']*d1['W']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#второй датасет, Т=1
d2 = ds2[['y_1','W']]
d2.columns = ['Y','W']
d2['T'] = 1
d2['WT'] = d2['T']*d2['W']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
did_ds = pd.concat([d1,d2])

In [ ]:
did_ds.head()

,Y,W,T,WT
0,2.785060,1.0,0,0.0
1,5.533515,0.0,0,0.0
2,0.095546,1.0,0,0.0
3,3.590472,1.0,0,0.0
4,1.365405,0.0,0,0.0


In [ ]:
formula = 'Y ~ C(T) + C(W) + C(WT)'

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
did = smf.ols(formula=formula, data=did_ds).fit()
did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     676.8
Date:                Wed, 26 May 2021   Prob (F-statistic):               0.00
Time:                        15:19:58   Log-Likelihood:                -43972.
No. Observations:               20000   AIC:                         8.795e+04
Df Residuals:                   19996   BIC:                         8.798e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8162      0.032     25.820      0.000       0.754       0.878
C(T)[T.1]        1.0624      0.045     23.764      0.000       0.975       1.150
C(W)[T.1.0]     -0.1237      0.044     -2.832      0.005      -0.209      -0.038
C(WT)[T.1.0]     0.5549      0.062      8.985      0.000       0.434       0.676
==============================================================================
Omnibus:                        0.070   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.966   Jarque-Bera (JB):                0.059
Skew:                          -0.002   Prob(JB):                        0.971
Kurtosis:                       3.007   Cond. No.                         7.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
ATE_DiD = 0.5549 # коэф перед произведением двух индикаторов T=1 W=1

2.2) CUPED

Нахожу параметр тетта

In [ ]:
teta = (np.cov(ds2['y_1'], ds2['y_0'])/np.var(ds2['y_0']))[0][1] # на главной диагонали ковариация себя с собой, поэтому нужно взять ячейку первой строки второго столбца

In [ ]:
teta

0.02361229032310578

In [ ]:
tau_cuped = ds2['y_1'] - (ds2['y_0']-ds2['y_0'].mean())*teta

Не уверен,что этот шаг я делаю правильно, но я повторяю принцип метода DiD, только вместо y_1 там будет tau_cuped. Почему я так сделал? В лекции говорилось,что кьюпед - обобщение дид, которое работает без допущения о паралелльности трендов. 

In [ ]:
ds_cuped = ds2.copy(deep = True)
ds_cuped['tau_cuped'] = tau_cuped

In [ ]:
#первый датасет, Т=0
d_c1 = ds_cuped[['y_0','W']]
d_c1.columns = ['Y','W']
d_c1['T'] = 0
d_c1['WT'] = d_c1['T']*d_c1['W']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#первый датасет, Т=1
d_c2 = ds_cuped[['tau_cuped','W']]
d_c2.columns = ['Y','W']
d_c2['T'] = 1
d_c2['WT'] = d_c2['T']*d_c2['W']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
cuped_ds = pd.concat([d_c1,d_c2])

In [ ]:
formula = 'Y ~ C(T) + C(W) + C(WT)'
cuped_did = smf.ols(formula=formula, data=cuped_ds).fit()
cuped_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     677.5
Date:                Wed, 26 May 2021   Prob (F-statistic):               0.00
Time:                        15:19:51   Log-Likelihood:                -43968.
No. Observations:               20000   AIC:                         8.794e+04
Df Residuals:                   19996   BIC:                         8.798e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8162      0.032     25.825      0.000       0.754       0.878
C(T)[T.1]        1.0608      0.045     23.734      0.000       0.973       1.148
C(W)[T.1.0]     -0.1237      0.044     -2.832      0.005      -0.209      -0.038
C(WT)[T.1.0]     0.5578      0.062      9.034      0.000       0.437       0.679
==============================================================================
Omnibus:                        0.062   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.970   Jarque-Bera (JB):                0.052
Skew:                          -0.003   Prob(JB):                        0.974
Kurtosis:                       3.006   Cond. No.                         7.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
ATE_cuped = 0.5578 

Не уверен, что нужно было поступать так, но других предположений у меня не было. Получил результат близкий к Did.

2.3) ANCOVA

In [ ]:
ds_anc = ds2[['y_1','y_0','W']]

In [ ]:
ds_anc.columns = ['Y','y_0','W']
formula = 'Y ~ y_0 + C(W)'

In [ ]:
anc = smf.ols(formula, ds_anc).fit()
anc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     58.70
Date:                Wed, 26 May 2021   Prob (F-statistic):           4.54e-26
Time:                        15:19:11   Log-Likelihood:                -21431.
No. Observations:               10000   AIC:                         4.287e+04
Df Residuals:                    9997   BIC:                         4.289e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.8572      0.031     60.304      0.000       1.797       1.918
C(W)[T.1.0]     0.4345      0.041     10.512      0.000       0.353       0.516
y_0             0.0262      0.009      2.905      0.004       0.009       0.044
==============================================================================
Omnibus:                        2.348   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.309   Jarque-Bera (JB):                2.338
Skew:                           0.037   Prob(JB):                        0.311
Kurtosis:                       3.005   Cond. No.                         5.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
ATE_anc = 0.4345

3,4) Сравнение оценок и вывод.

In [ ]:
anc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     58.70
Date:                Wed, 26 May 2021   Prob (F-statistic):           4.54e-26
Time:                        15:20:03   Log-Likelihood:                -21431.
No. Observations:               10000   AIC:                         4.287e+04
Df Residuals:                    9997   BIC:                         4.289e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.8572      0.031     60.304      0.000       1.797       1.918
C(W)[T.1.0]     0.4345      0.041     10.512      0.000       0.353       0.516
y_0             0.0262      0.009      2.905      0.004       0.009       0.044
==============================================================================
Omnibus:                        2.348   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.309   Jarque-Bera (JB):                2.338
Skew:                           0.037   Prob(JB):                        0.311
Kurtosis:                       3.005   Cond. No.                         5.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     676.8
Date:                Wed, 26 May 2021   Prob (F-statistic):               0.00
Time:                        15:20:04   Log-Likelihood:                -43972.
No. Observations:               20000   AIC:                         8.795e+04
Df Residuals:                   19996   BIC:                         8.798e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8162      0.032     25.820      0.000       0.754       0.878
C(T)[T.1]        1.0624      0.045     23.764      0.000       0.975       1.150
C(W)[T.1.0]     -0.1237      0.044     -2.832      0.005      -0.209      -0.038
C(WT)[T.1.0]     0.5549      0.062      8.985      0.000       0.434       0.676
==============================================================================
Omnibus:                        0.070   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.966   Jarque-Bera (JB):                0.059
Skew:                          -0.002   Prob(JB):                        0.971
Kurtosis:                       3.007   Cond. No.                         7.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
cuped_did.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     677.5
Date:                Wed, 26 May 2021   Prob (F-statistic):               0.00
Time:                        15:20:14   Log-Likelihood:                -43968.
No. Observations:               20000   AIC:                         8.794e+04
Df Residuals:                   19996   BIC:                         8.798e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8162      0.032     25.825      0.000       0.754       0.878
C(T)[T.1]        1.0608      0.045     23.734      0.000       0.973       1.148
C(W)[T.1.0]     -0.1237      0.044     -2.832      0.005      -0.209      -0.038
C(WT)[T.1.0]     0.5578      0.062      9.034      0.000       0.437       0.679
==============================================================================
Omnibus:                        0.062   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.970   Jarque-Bera (JB):                0.052
Skew:                          -0.003   Prob(JB):                        0.974
Kurtosis:                       3.006   Cond. No.                         7.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Полученные оценки АТЕ методом did and cuped различаются незначительно (если я правильно нашел для кьюпеда :) ). Однако в данных методах СКО выше, чем при использовании АНКОВЫ. Вывод: оценка методом ANCOVA дает наименьший разброс.

### 3. Построение uplift модели по результатам AB теста.

Вам даны результаты AB теста для модели конверсии клиента в продукте (conversion). Конверсия может принимать значения 1 или 0, что значит - клиент подключил услугу, либо не подключил соответственно.

Задача:
1. Оцените ATE по данным.
2. Оцените CATE для каждого наблюдения.
3. Проставьте рекомендации к воздействию на каждый объект.
4. Какая ожидаемая конверсия при оптимальном воздействии?
5. Все выводы, предпосылки и методы представляйте в явном виде в ноутбуке.

Визуализация результатов также будет приветствоваться.

dataset_part3_students.csv

In [ ]:
ds3.head()

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion
0,control,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,-0.807218,1.318646,-0.065262,-0.956947,-0.314573,-0.879456,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0
1,control,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,-0.460971,-1.396720,0.600044,1.000000,0.303664,1.080643,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0
2,control,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,-1.430502,0.469694,1.251940,0.943003,-0.083721,-1.981712,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0
3,treatment,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,0.408691,-0.422736,-1.593407,0.247038,-0.752011,-0.347319,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1
4,control,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,-0.243256,0.598662,0.750706,0.554053,-1.198151,-0.232961,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0


Бинаризую категориальный признак treatment

In [ ]:
ds3['treatment'].replace({'control': 0, 'treatment': 1}, inplace=True)


In [ ]:
ds3.head()

,treatment,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,conversion
0,0,0.034351,0.550725,-1.348524,-0.029591,0.499535,-0.412464,0.933588,-0.522954,-0.029591,-0.807218,1.318646,-0.065262,-0.956947,-0.314573,-0.879456,0.640264,1.165722,0.570495,0.529443,0.227962,1.929914,0.798664,0
1,0,-1.059404,0.299404,0.220273,-1.569660,1.084204,-2.167118,1.216469,-0.744547,-1.569660,-0.460971,-1.396720,0.600044,1.000000,0.303664,1.080643,2.646483,-0.329690,1.725636,1.434649,-2.064893,0.188000,-1.134651,0
2,0,0.051934,2.338729,-0.341017,-1.099362,0.693090,-2.424267,1.572066,-0.212798,-1.099362,-1.430502,0.469694,1.251940,0.943003,-0.083721,-1.981712,0.224777,2.038004,2.516398,2.324280,-3.182196,-0.888490,-1.402561,0
3,1,0.667770,0.539316,0.468719,-0.840885,-0.708331,0.126634,1.275854,-0.954919,-0.840885,0.408691,-0.422736,-1.593407,0.247038,-0.752011,-0.347319,-1.659402,-0.871980,-0.893513,-0.658078,-2.704997,-0.390745,-1.578004,1
4,0,-1.416158,2.486628,0.496402,-1.174618,2.738267,-4.756755,-0.180918,-1.281432,-1.174618,-0.243256,0.598662,0.750706,0.554053,-1.198151,-0.232961,0.948875,1.210523,1.536723,1.548610,-0.926435,0.972459,-0.711251,0


1) Оценка ATE

Найду АТЕ как коэф перед W в модели линейной регрессии.


In [ ]:
model3 = LinearRegression()
X3 = ds3.drop('conversion', axis=1)
y3 = ds3['conversion']
model3.fit(X3, y3)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
ATE_3 = model3.coef_[0]
ATE_3

0.06024252397448968

2) Оценка CATE

2.1) S learner

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_w0 = ds3.drop('conversion',axis =1)
X_w0['treatment'] = 0

In [ ]:
X_w1 = ds3.drop('conversion',axis =1)
X_w1['treatment'] = 1

Дальше не доделал.